In [1]:
# Load model directly
from transformers import AutoTokenizer
import pandas as pd
import polars as pl

tokenizer_zinc = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")

/home/visanim/.cache/pypoetry/virtualenvs/lotus-hypersketching-ruHkzsWs-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
lotus = pd.read_csv("./data/molecules/230106_frozen_metadata.csv.gz", low_memory=False)

In [3]:
lotus["wd_molecule"] = "wd:" + lotus.structure_wikidata.str.extract(r"(Q\d+)")


wd = lotus[
    [
        "wd_molecule",
        "structure_smiles_2D",
    ]
].drop_duplicates()

In [4]:
embedding = {
    row["wd_molecule"]: tokenizer_zinc(
        row["structure_smiles_2D"],
        padding="max_length",
        # max_length=256,
        return_tensors="np",
    )["input_ids"].flatten()
    for index, row in wd.iterrows()
}

In [5]:
df = pl.from_dict(embedding).transpose(include_header=True).to_pandas()

ShapeError: could not create a new DataFrame: series "wd:Q43656" has length 256 while series "wd:Q425134" has length 268

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)

pca_fitted = pca.fit_transform(df)

In [ ]:
pca_fitted = pd.DataFrame(pca_fitted, index=df.index)
pca_fitted_sub_sample = pca_fitted.sample(10000, random_state=42)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(
    n_components=2,
    verbose=2,
    random_state=42,
)

tsne_fitted = tsne.fit_transform(pca_fitted_sub_sample)


import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 10))
color = (
    lotus[lotus.structure_smiles_2D.isin(pca_fitted_sub_sample.index)]
    .drop_duplicates(subset="structure_smiles_2D")
    .structure_xlogp
)

sns.scatterplot(x=tsne_fitted[:, 0], y=tsne_fitted[:, 1], hue=color, palette="viridis")

In [ ]:
color = lotus[lotus.structure_smiles_2D.isin(pca_fitted_sub_sample.index)]

In [ ]:
color.drop_duplicates(subset="structure_smiles_2D", inplace=True)

In [ ]:
lotus.columns